# M1 — Fundamentos: ONNX + ONNX Runtime (y dónde encaja Olive)

## Objetivo
En este laboratorio vas a:

1. Crear un modelo **ONNX** mínimo (un grafo que calcula `Y = X + 1`).
2. Guardarlo en `models/`.
3. Ejecutarlo con **ONNX Runtime** (ORT) desde Python.
4. Inspeccionar **Execution Providers** disponibles y verificar el resultado.

## Prerrequisitos
- Estar usando el **kernel** correcto en VS Code (tu `.venv`).
- Tener instalados: `numpy`, `onnx`, `onnxruntime`.

> Nota: si alguna importación falla, usa la celda de instalación (más abajo) y vuelve a ejecutar.


## 0) Estructura recomendada del proyecto

```
notebooks/
models/
outputs/
cache/        (opcional, para M2+)
requirements.txt (opcional)
```


In [1]:
from pathlib import Path

# Crear carpetas al mismo nivel que notebooks
Path("../models").mkdir(exist_ok=True)
Path("../outputs").mkdir(exist_ok=True)

print("OK: carpetas creadas/ya existen fuera de notebooks")


OK: carpetas creadas/ya existen fuera de notebooks


## 1) Chequeo de entorno (Python, paquetes, providers)

Esta celda te ayuda a confirmar que:
- El notebook está ejecutándose con el Python del entorno correcto.
- Qué versiones tienes instaladas.
- Qué providers (EPs) están disponibles en tu ORT.


In [2]:
import sys, subprocess

print("Python executable:", sys.executable)
print("Python version:", sys.version)

for pkg in ["onnx", "onnxruntime", "numpy", "olive-ai"]:
    print(f"\n--- pip show {pkg} ---")
    subprocess.run([sys.executable, "-m", "pip", "show", pkg], check=False)

print("\n--- pip check ---")
subprocess.run([sys.executable, "-m", "pip", "check"], check=False)


Python executable: c:\source\VisualCode\repos\olive-python-vscode-labs\.venv\Scripts\python.exe
Python version: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]

--- pip show onnx ---

--- pip show onnxruntime ---

--- pip show numpy ---

--- pip show olive-ai ---

--- pip check ---


CompletedProcess(args=['c:\\source\\VisualCode\\repos\\olive-python-vscode-labs\\.venv\\Scripts\\python.exe', '-m', 'pip', 'check'], returncode=0)

## 2) (Opcional) Instalar dependencias si faltan

Ejecuta esta celda **solo si** en la celda anterior viste errores de importación o paquetes ausentes.


In [3]:
import sys, subprocess

# Si ya lo tienes, pip dirá "Requirement already satisfied"
subprocess.run([sys.executable, "-m", "pip", "install", "numpy", "onnx", "onnxruntime"], check=False)


CompletedProcess(args=['c:\\source\\VisualCode\\repos\\olive-python-vscode-labs\\.venv\\Scripts\\python.exe', '-m', 'pip', 'install', 'numpy', 'onnx', 'onnxruntime'], returncode=0)

## 3) Crear un modelo ONNX mínimo (Y = X + 1)

Vamos a construir un grafo ONNX con:
- 1 entrada `X` (vector 1D de float32, longitud variable)
- 1 constante `C = [1.0]`
- 1 nodo `Add(X, C) -> Y`

### Compatibilidad (IR / opset)
A veces pueden aparecer errores de compatibilidad (por ejemplo, “Unsupported model IR version”).
Para evitarlo, fijamos explícitamente:
- `ir_version = 11`
- `opset = 11`

Si tu runtime soporta versiones más altas, esto seguirá funcionando.


In [4]:
from pathlib import Path
import numpy as np
import onnx
from onnx import TensorProto, helper, numpy_helper

# Ruta de salida (coincide con la celda que crea ../models)
model_path = Path("../models") / "add_const.onnx"
model_path.parent.mkdir(parents=True, exist_ok=True)

# IO
X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [None])  # vector 1D tamaño variable
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [None])

# Constante C = [1.0]
C_value = np.array([1.0], dtype=np.float32)
C = numpy_helper.from_array(C_value, name="C")

# Nodo Add
node = helper.make_node("Add", inputs=["X", "C"], outputs=["Y"])

# Grafo
graph = helper.make_graph(
    nodes=[node],
    name="add-const",
    inputs=[X],
    outputs=[Y],
    initializer=[C],
)

# Modelo (fijando opset e ir_version para compatibilidad)
opset = [helper.make_operatorsetid("", 11)]
model = helper.make_model(
    graph,
    producer_name="m1-lab",
    opset_imports=opset,
    ir_version=11,
)

# Validar + guardar
onnx.checker.check_model(model)
onnx.save_model(model, str(model_path))

print("Saved:", model_path)
print("Model ir_version:", model.ir_version)
print("Model opsets:", [(o.domain, o.version) for o in model.opset_import])


Saved: ..\models\add_const.onnx
Model ir_version: 11
Model opsets: [('', 11)]


## 4) Ejecutar inferencia con ONNX Runtime

Cargamos el modelo con `InferenceSession` y ejecutamos con un input `x`.


In [5]:
import numpy as np
import onnxruntime as ort

print("ORT version:", ort.get_version_string())
print("Available providers:", ort.get_available_providers())

# Cargar modelo y forzar CPU (reproducible)
sess = ort.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])

input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

x = np.array([10, 20, 30], dtype=np.float32)
y = sess.run([output_name], {input_name: x})[0]

print("input_name:", input_name, "| output_name:", output_name)
print("x:", x)
print("y:", y)


ORT version: 1.23.2
Available providers: ['AzureExecutionProvider', 'CPUExecutionProvider']
input_name: X | output_name: Y
x: [10. 20. 30.]
y: [11. 21. 31.]


## 5) Inspección rápida del modelo ONNX (grafo, nodos, inicializadores)

Esto te ayuda a “ver” qué contiene el archivo `.onnx`.


In [6]:
import onnx

m = onnx.load(str(model_path))
print("ir_version:", m.ir_version)
print("opset_imports:", [(o.domain, o.version) for o in m.opset_import])

print("\nGraph:", m.graph.name)
print("Inputs:", [i.name for i in m.graph.input])
print("Outputs:", [o.name for o in m.graph.output])
print("Initializers:", [t.name for t in m.graph.initializer])

print("\nNodes:")
for n in m.graph.node:
    print("-", n.op_type, "inputs=", list(n.input), "outputs=", list(n.output))


ir_version: 11
opset_imports: [('', 11)]

Graph: add-const
Inputs: ['X']
Outputs: ['Y']
Initializers: ['C']

Nodes:
- Add inputs= ['X', 'C'] outputs= ['Y']


## Verificación (checklist)

- `models/add_const.onnx` existe.
- La inferencia imprime:
  - `x: [10. 20. 30.]`
  - `y: [11. 21. 31.]`
- `Available providers` incluye `CPUExecutionProvider`.

Si esto pasa, M1 está completado ✅


## Errores comunes y diagnóstico

### 1) `ModuleNotFoundError`
- Asegúrate de estar en el kernel correcto (tu `.venv`).
- Ejecuta la celda **2) Instalar dependencias** y reinicia el kernel.

### 2) `Unsupported model IR version ...`
- Mantén `ir_version=11` y `opset=11` como está en la celda 3.
- Si quieres usar versiones más nuevas, actualiza `onnxruntime`.

### 3) El modelo corre pero no ves providers GPU/NPU
- `ort.get_available_providers()` lista lo que tienes realmente instalado en ese entorno.
- Para GPU/NPU normalmente necesitas un paquete de ORT/EP específico para tu hardware.


## Siguiente paso (M2)

En M2 haremos un workflow real de **Olive** (YAML/JSON) para:
- tomar un modelo ONNX
- aplicar passes de optimización
- generar artefactos en `outputs/`
- validar antes/después con ORT
